In [501]:
import numpy as np
import pandas as pd

In [502]:
# Load MutPred2 predictions
mutpred_predictions = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\MutPred2\\all_predictions.csv", header=None)
mutpred_predictions

In [503]:
# add headers
column_names = ["gene", "variant", "MutPred_score", "explain", "MutPred_pred", "nothing"]
mutpred_predictions.columns = column_names
mutpred_predictions = mutpred_predictions.drop(columns=["nothing", "explain", "MutPred_pred"])
mutpred_predictions

In [504]:
# Delete the _HUMAN from the gene name
mutpred_predictions["gene"] = mutpred_predictions["gene"].str.replace("_HUMAN", "")
print(len(mutpred_predictions))

,chr,pos_hg19,pos_grch38,ref_na,alt_na,aa_wt,aa_mut,revel_score,transcript_id,aa_pos,gene
0,12,48398104,48004321,T,G,M,L,0.562,ENST00000380518;ENST00000395281;ENST00000337299,1,COL2A1
1,12,48398104,48004321,T,C,M,V,0.585,ENST00000380518;ENST00000395281;ENST00000337299,1,COL2A1
2,12,48398104,48004321,T,A,M,L,0.57,ENST00000380518;ENST00000395281;ENST00000337299,1,COL2A1
3,12,48398103,48004320,A,T,M,K,0.676,ENST00000380518;ENST00000395281;ENST00000337299,1,COL2A1
4,12,48398103,48004320,A,G,M,T,0.691,ENST00000380518;ENST00000395281;ENST00000337299,1,COL2A1
...,...,...,...,...,...,...,...,...,...,...,...
112508,4,6304190,6302463,G,C,A,P,0.352,ENST00000503569;ENST00000226760;ENST00000540337,890,WFS1
112509,4,6304190,6302463,G,T,A,S,0.224,ENST00000503569;ENST00000226760;ENST00000540337,890,WFS1
112510,4,6304191,6302464,C,A,A,D,0.435,ENST00000503569;ENST00000226760;ENST00000540337,890,WFS1
112511,4,6304191,6302464,C,G,A,G,0.159,ENST00000503569;ENST00000226760;ENST00000540337,890,WFS1


In [505]:

# remove duplicates
mutpred_predictions = mutpred_predictions.drop_duplicates(subset=["gene", "variant"], keep="first")
print(len(mutpred_predictions))

C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_11864\1654159982.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  REVEL_predictions["aa_pos"] = REVEL_predictions["aa_pos"].astype(str)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_11864\1654159982.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  REVEL_predictions['variant'] = REVEL_predictions['aa_wt'] + REVEL_predictions['aa_pos'] + REVEL_predictions['aa_mut']


,revel_score,gene,variant
0,0.562,COL2A1,M1L
1,0.585,COL2A1,M1V
2,0.57,COL2A1,M1L
3,0.676,COL2A1,M1K
4,0.691,COL2A1,M1T
...,...,...,...
112508,0.352,WFS1,A890P
112509,0.224,WFS1,A890S
112510,0.435,WFS1,A890D
112511,0.159,WFS1,A890G


In [506]:
# Load VEST4 predictions
VEST4_predictions = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\VEST4\\all_vest4_predictions.csv")
VEST4_predictions = VEST4_predictions.drop(columns=["S.O. transcript", "Amino acid position"])

# rename columns
# Change name of column "HUGO symbol" to "gene"
VEST4_predictions = VEST4_predictions.rename(columns={"HUGO symbol": "gene"})
# Change name of column "Protein sequence change" to "variant"
VEST4_predictions = VEST4_predictions.rename(columns={"Protein sequence change": "variant"})
# Change name of column "VEST4 score (missense)" to "VEST4_score"
VEST4_predictions = VEST4_predictions.rename(columns={"VEST score (missense)": "VEST_score"})
VEST4_predictions

In [507]:
# Combine the prediction files into one file
my_prediction = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\all_inbar_predictions.csv")
my_prediction



In [508]:
# Merge the two files
merged = pd.merge(my_prediction, mutpred_predictions, on=["gene", "variant"])
merged = pd.merge(merged, VEST4_predictions, on=["gene", "variant"])
merged

In [509]:
print(f"length of merged {len(merged)}")
print(f"length of my_prediction {len(my_prediction)}")
print(f"length of mutpred {len(mutpred_predictions)}")

In [510]:
# # Save the merged file to csv
# merged.to_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\merged_my_prediction_and_mutpred.csv", index=False)

In [511]:
## Calculate the MCC for the mutpred predictions, according to the real pathogenicity (0 or 1)
from sklearn.metrics import matthews_corrcoef

,position,pathogenicity,predictions,variant,gene
0,6,0,0,G6S,SLC26A4
1,21,0,1,M21V,SLC26A4
2,51,0,0,A51T,SLC26A4
3,53,0,1,C53G,SLC26A4
4,163,0,1,V163I,SLC26A4
...,...,...,...,...,...
4516,1045,1,1,G1045V,COL4A3
4517,871,1,1,G871C,COL4A3
4518,853,1,1,G853R,COL4A3
4519,25,0,0,P25S,COL4A3


In [512]:
# separate the merged dataframe according to gene
genes = merged["gene"].unique()
print(f"Number of genes: {len(genes)}")

In [513]:
# Calculate MCC for each gene specific predictor for mutpred

# Build dictionary with gene names as keys.
mccs = {gene: 0 for gene in genes}

for gene in genes:
    gene_df = merged[merged["gene"] == gene]
    # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
    # Create binary predictions based on the 0.5 threshold
    gene_df.loc[:, 'binary_prediction_mutpred'] = (gene_df['MutPred_score'] > 0.5).astype(int)
    
    gene_df['predictions'] = gene_df['predictions'].astype(int)
    
    # Calculate MCC
    mcc = matthews_corrcoef(gene_df['predictions'], gene_df['binary_prediction_mutpred'])
    
    # Get gene name for the use for the dictionary
    gene = gene_df['gene'].unique()[0]
    
    # Append mcc to dictionary
    mccs[gene] = mcc

print("MCCs of MutPred2 predictions for each gene:")
for gene in mccs:
    print(f"{gene}: {mccs[gene]}")


In [514]:
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score, f1_score, log_loss, confusion_matrix
# store the results in a dictionary
results = {gene: 0 for gene in genes}

for gene in genes:
    gene_df = merged[merged["gene"] == gene]
    y_true = gene_df["pathogenicity"].astype(int)
    predicted_probabilities = gene_df["MutPred_score"]
    gene_df.loc[:, 'binary_prediction_mutpred'] = (gene_df['MutPred_score'] > 0.5).astype(int)
    predicted_labels = gene_df["binary_prediction_mutpred"]
    
    results_gene = {}
    # Calculate AUC-ROC
    roc_auc = roc_auc_score(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["roc_auc"] = roc_auc
    
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, predicted_probabilities)
    roc_auc = auc(fpr, tpr)
    # append to dictionary
    results_gene["auc"] = roc_auc
    
    # Calculate AUC-PR
    precision, recall, _ = precision_recall_curve(y_true, predicted_probabilities)
    pr_auc = auc(recall, precision)
    # append to dictionary
    results_gene["pr_auc"] = pr_auc
    
    # Calculate F1 Score
    f1 = f1_score(y_true, predicted_labels)
    # append to dictionary
    results_gene["f1"] = f1
    
    # Calculate Log Loss
    logloss = log_loss(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["logloss"] = logloss
    
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(y_true, predicted_labels)
    # append to dictionary
    results_gene["confusion_matrix"] = conf_matrix
    
    # Get gene name for the use for the dictionary
    gene = gene_df['gene'].unique()[0]
    # Append results to dictionary
    results[gene] = results_gene
    
## Print results
print("Results of MutPred2 predictions for each gene:")
for gene in results:
    print(f"{gene}: {results[gene]}")

,position,pathogenicity,predictions,variant,gene,revel_score
0,7,0,1,P7Q,WFS1,0.178
1,16,0,0,P16L,WFS1,0.142
2,54,0,0,V54G,WFS1,0.409
3,57,0,0,A57T,WFS1,0.164
4,66,0,0,Q66H,WFS1,0.243
...,...,...,...,...,...,...
3978,853,1,1,G853R,COL4A3,0.988
3979,25,0,0,P25S,COL4A3,0.172
3980,1209,0,0,M1209I,COL4A3,0.187
3981,1209,0,0,M1209I,COL4A3,0.187


In [515]:
# present the confusion matrix for each gene
import matplotlib.pyplot as plt

for gene in results:
    # Plot confusion matrix
    plt.imshow(results[gene]["confusion_matrix"], cmap=plt.cm.Blues)
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.xticks([0, 1], ["Benign", "Pathogenic"])
    plt.yticks([0, 1], ["Benign", "Pathogenic"])
    plt.title(f"Confusion matrix for {gene}, MutPred2 predictions")
    plt.style.use("seaborn-white")
    # Add text annotations
    for i in range(2):
        for j in range(2):
            plt.text(j, i, results[gene]["confusion_matrix"][i, j], ha="center", va="center", color="black", backgroundcolor="white")
    plt.colorbar()
    plt.show()

In [516]:
# present the ROC curve for each gene
for gene in results:
    # Plot ROC curve
    plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {results[gene]['roc_auc']:.2f})")
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC curve for {gene}, MutPred2 predictions")
    plt.legend(loc="lower right")
    plt.show()

In [517]:
# Calculate MCC for each gene specific predictor for mutpred

# Build dictionary with gene names as keys.
mccs = {gene: 0 for gene in genes}

merged['VEST_score'] = merged['VEST_score'].astype(float)

for gene in genes:
    gene_df = merged[merged["gene"] == gene]
    # change VEST score to float
    # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
    # Create binary predictions based on the 0.5 threshold
    gene_df.loc[:, 'binary_prediction_VEST4'] = (gene_df['VEST_score'] > 0.5).astype(int)
    
    gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
    
    # Calculate MCC
    mcc = matthews_corrcoef(gene_df['pathogenicity'], gene_df['binary_prediction_VEST4'])
    
    # Get gene name for the use for the dictionary
    gene = gene_df['gene'].unique()[0]
    
    # Append mcc to dictionary
    mccs[gene] = mcc

print("MCCs of VEST4 predictions for each gene:")
for gene in mccs:
    print(f"{gene}: {mccs[gene]}")

Number of genes: 6


In [518]:
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score, f1_score, log_loss, confusion_matrix
# store the results in a dictionary
results = {gene: 0 for gene in genes}

for gene in genes:
    gene_df = merged[merged["gene"] == gene]
    y_true = gene_df["pathogenicity"].astype(int)
    predicted_probabilities = gene_df["MutPred_score"]
    gene_df.loc[:, 'binary_prediction_VEST4'] = (gene_df['VEST_score'] > 0.5).astype(int)
    predicted_labels = gene_df["binary_prediction_VEST4"]
    
    results_gene = {}
    # Calculate AUC-ROC
    roc_auc = roc_auc_score(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["roc_auc"] = roc_auc
    
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, predicted_probabilities)
    roc_auc = auc(fpr, tpr)
    # append to dictionary
    results_gene["auc"] = roc_auc
    
    # Calculate AUC-PR
    precision, recall, _ = precision_recall_curve(y_true, predicted_probabilities)
    pr_auc = auc(recall, precision)
    # append to dictionary
    results_gene["pr_auc"] = pr_auc
    
    # Calculate F1 Score
    f1 = f1_score(y_true, predicted_labels)
    # append to dictionary
    results_gene["f1"] = f1
    
    # Calculate Log Loss
    logloss = log_loss(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["logloss"] = logloss
    
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(y_true, predicted_labels)
    # append to dictionary
    results_gene["confusion_matrix"] = conf_matrix
    
    # Get gene name for the use for the dictionary
    gene = gene_df['gene'].unique()[0]
    # Append results to dictionary
    results[gene] = results_gene
    
## Print results
print("Results of VEST4 predictions for each gene:")
for gene in results:
    print(f"{gene}: {results[gene]}")

MCCs of REVEL predictions for each gene:
WFS1: 0.3508853905797042
FGFR1: 0.4047176158495665
COL2A1: 0.5061954051761961
COL4A5: 0.24804682599789998
MYO7A: 0.23102454281119714
COL4A3: 0.4878116751740154


C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_11864\3022861317.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['revel_score'] = gene_df['revel_score'].astype(float)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_11864\3022861317.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df.loc[:, 'binary_prediction_REVEL'] = (gene_df['revel_score'] > 0.5).astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_11864\3022861317.py:42: SettingWithCopyWarning: 
A value is trying to be set

In [519]:
# present the confusion matrix for each gene
import matplotlib.pyplot as plt

for gene in results:
    # Plot confusion matrix
    plt.imshow(results[gene]["confusion_matrix"], cmap=plt.cm.Blues)
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.xticks([0, 1], ["Benign", "Pathogenic"])
    plt.yticks([0, 1], ["Benign", "Pathogenic"])
    plt.title(f"Confusion matrix for {gene}, VEST4 predictions")
    plt.style.use("seaborn-white")
    # Add text annotations
    for i in range(2):
        for j in range(2):
            plt.text(j, i, results[gene]["confusion_matrix"][i, j], ha="center", va="center", color="black", backgroundcolor="white")
    plt.colorbar()
    plt.show()

In [520]:
#present the ROC curve for each gene
for gene in results:
    # Plot ROC curve
    plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {results[gene]['roc_auc']:.2f})")
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC curve for {gene}, VEST4 predictions")
    plt.legend(loc="lower right")
    plt.show()